In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import EoN

In [ ]:
def download_file(url):
    file_path = url.split('/')[-1]
    resp = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(resp.content)
    return file_path

# 1. Epidemias e marketing viral

## Tempo para propagação

In [ ]:
G = nx.path_graph(5)
nx.draw_networkx(G)

In [ ]:
nx.eccentricity(G)

O **centro** da rede indica os nós que propagam informações rapidamente:

In [ ]:
nx.center(G)

In [ ]:
G = nx.barbell_graph(6, 2)
nx.draw_networkx(G)

In [ ]:
nx.center(G)

In [ ]:
nx.eccentricity(G, 6)

## Modelo de epidemia
Biblioteca `EoN` (*Epidemics on Networks*)

https://epidemicsonnetworks.readthedocs.io/en/latest/EoN.html

In [ ]:
pip install EoN --quiet

In [ ]:
N = 10000
p = 0.01
G = nx.erdos_renyi_graph(N, p, seed=42)
print(G)

### Parâmetros

In [ ]:
tmax = 10
tau = 0.1  # transmission rate
gamma = 1.0  # recovery rate
rho = 0.005  # random fraction initially infected

### Modelo com imunidade

In [ ]:
t, S, I, R = EoN.fast_SIR(G, tau, gamma, rho=rho, tmax=tmax)

plt.plot(t, I)
plt.xlabel('Tempo')
plt.ylabel('Número de infectados');

### Modelo **sem** imunidade

In [ ]:
t, S, I = EoN.fast_SIS(G, tau, gamma, rho=rho, tmax=tmax)

plt.plot(t, I)
plt.xlabel('Tempo')
plt.ylabel('Número de infectados');

# 2. Busca em redes

In [ ]:
url = 'https://snap.stanford.edu/data/roadNet-PA.txt.gz'
file_path = download_file(url)
file_path

Lembrando que a rede é direcionada!

In [ ]:
G = nx.read_edgelist(file_path, create_using=nx.DiGraph)
print(G)

In [ ]:
np.random.seed(42)
for src, dst in G.edges:
    G.edges[src, dst]['road_length'] = np.random.randint(1, 50)

## Busca sem pesos

Caminhos até os vizinhos:

In [ ]:
G.adj['0']

In [ ]:
nx.shortest_path(G, '0', '1')

In [ ]:
nx.shortest_path(G, '0', '6309')

In [ ]:
nx.shortest_path(G, '0', '6353')

Para saber a distância:

In [ ]:
nx.shortest_path_length(G, '0', '6353')

Caminhos de comprimento 2:

In [ ]:
G.adj['6309']

In [ ]:
nx.shortest_path(G, '0', '6310')

In [ ]:
nx.shortest_path_length(G, '0', '6310')

Distâncias podem crescer rapidamente nessa rede:

In [ ]:
nx.shortest_path(G, '0', '2')

In [ ]:
nx.shortest_path_length(G, '0', '2')

In [ ]:
path = nx.shortest_path(G, '0', '2')
len(path) - 1

## Busca com pesos

In [ ]:
nx.shortest_path(G, '0', '1', weight='road_length')

In [ ]:
nx.shortest_path_length(G, '0', '1', weight='road_length')

In [ ]:
path_to_2 = nx.shortest_path(G, '0', '2', weight='road_length')
len(path_to_2)

In [ ]:
nx.shortest_path_length(G, '0', '2', weight='road_length')

# 3. Falhas em redes

## Rede aleatória *Erdös-Rényi*

In [ ]:
n = 5000
p = 0.005
G = nx.erdos_renyi_graph(n, p, seed=42)
print(G)

No início, a rede está conectada:

In [ ]:
nx.is_connected(G)

In [ ]:
def attack(g, targeted, seed=None):
    np.random.seed(seed)  # Para fins didáticos, vamos fixar a semente
    num_removed = 0
    while nx.is_connected(g):
        if not targeted:  # Escolha totalmente aleatória
            node_to_remove = np.random.choice(g.nodes)
        else:  # Escolha proporcional ao grau
            degrees = [degree for node, degree in g.degree]
            degrees = np.array(degrees)
            prob = degrees / np.sum(degrees)  # Normalizando para somar 1
            node_to_remove = np.random.choice(g.nodes, p=prob)
        g.remove_node(node_to_remove)
        num_removed += 1
    print(f'Nós removidos até desconectar: {num_removed}')

In [ ]:
H = G.copy()
attack(H, targeted=False, seed=42)

In [ ]:
print(H)

In [ ]:
nx.is_connected(H)

In [ ]:
H = G.copy()
attack(H, targeted=True, seed=42)

In [ ]:
print(H)

## Rede livre de escala *Barabási-Albert*

In [ ]:
G = nx.barabasi_albert_graph(n, 2, seed=42)
print(G)

In [ ]:
nx.is_connected(G)

In [ ]:
H = G.copy()
attack(H, targeted=False, seed=42)

In [ ]:
print(H)

In [ ]:
nx.is_connected(H)

In [ ]:
H = G.copy()
attack(H, targeted=True, seed=42)

In [ ]:
print(H)

In [ ]:
H = G.copy()
attack(H, targeted=False, seed=1)

In [ ]:
H = G.copy()
attack(H, targeted=True, seed=1)

# 4. Recomendação em redes

**Rede de colaboração**

- Artigos na área de astrofísica
- Publicados entre 1993 e 2003

[Link para o dataset](https://snap.stanford.edu/data/ca-AstroPh.html)

Referência:
- J. Leskovec, J. Kleinberg and C. Faloutsos. Graph Evolution: Densification and Shrinking Diameters. ACM Transactions on Knowledge Discovery from Data (ACM TKDD), 1(1), 2007.

In [80]:
url = 'https://snap.stanford.edu/data/ca-AstroPh.txt.gz'
file_path = download_file(url)
file_path

'ca-AstroPh.txt.gz'

In [81]:
G = nx.read_edgelist(file_path)
print(G)

Graph with 18772 nodes and 198110 edges


Algumas opções para prever relações:
1. `nx.common_neighbor_centrality()`
2. `nx.jaccard_coefficient`
3. `nx.adamic_adar_index`
4. `nx.preferential_attachment`

Nosso nó de exemplo:

In [82]:
target_node = '276'
G.degree[target_node]

31

In [83]:
pairs = [(target_node, '283')]
recommendation = nx.common_neighbor_centrality(G, pairs, alpha=1.0)
recommendation = list(recommendation)
recommendation

[('276', '283', 1)]

In [84]:
list(nx.non_neighbors(G, target_node))[:10]

['1662',
 '5089',
 '6058',
 '6229',
 '10639',
 '16442',
 '19325',
 '19834',
 '21937',
 '25452']

In [85]:
available_recs = [(target_node, n) for n in nx.non_neighbors(G, target_node)]

## Vizinhos em comum

In [86]:
recommendation = nx.common_neighbor_centrality(G, available_recs, alpha=1.0)
recommendation = pd.DataFrame(
    recommendation,
    columns=['source', 'destination', 'prediction']
)
recommendation.sort_values('prediction', ascending=False)

,source,destination,prediction
34,276,72391,4
39,276,78627,4
13,276,32432,3
37,276,77098,3
6256,276,15309,3
...,...,...,...
6549,276,93404,0
6550,276,95461,0
6551,276,96331,0
6552,276,96364,0


## Coeficiente de *Jaccard*

In [87]:
recommendation2 = nx.jaccard_coefficient(G, available_recs)
recommendation2 = pd.DataFrame(
    recommendation2,
    columns=['source', 'destination', 'prediction']
)
recommendation2.sort_values('prediction', ascending=False)

,source,destination,prediction
34,276,72391,0.117647
12704,276,3539,0.090909
8486,276,132043,0.073171
3303,276,124591,0.064516
1,276,5089,0.064516
...,...,...,...
6549,276,93404,0.000000
6550,276,95461,0.000000
6551,276,96331,0.000000
6552,276,96364,0.000000


## *Adamic-Adar*

In [88]:
recommendation3 = nx.adamic_adar_index(G, available_recs)
recommendation3 = pd.DataFrame(
    recommendation3,
    columns=['source', 'destination', 'prediction']
)
recommendation3.sort_values('prediction', ascending=False)

,source,destination,prediction
2140,276,120425,1.338287
34,276,72391,1.068811
39,276,78627,1.062724
12704,276,3539,0.993491
12694,276,10110,0.993491
...,...,...,...
6549,276,93404,0.000000
6550,276,95461,0.000000
6551,276,96331,0.000000
6552,276,96364,0.000000


## Conexão preferencial

In [89]:
recommendation4 = nx.preferential_attachment(G, available_recs)
recommendation4 = pd.DataFrame(
    recommendation4,
    columns=['source', 'destination', 'prediction']
)
recommendation4.sort_values('prediction', ascending=False)

,source,destination,prediction
315,276,53213,15624
288,276,35290,13237
293,276,38109,13020
2284,276,62821,12958
2115,276,93504,11997
...,...,...,...
17578,276,87910,31
17577,276,117290,31
17576,276,95148,31
5242,276,95880,31


In [90]:
G.degree['53213'], G.degree['35290']

(504, 427)